In [124]:
from pathlib import Path
import networkx as nx
import re
import sys

input_file = Path(".") / "example2_p1-2_p2-9.txt"

class Indicator:    
    def __init__(self, lights_count, lights_state: int):
        '''
        Args:
            lights (str): Represents state of lights in binary string format. For example: four lights, first on, three off: "1000"
        '''
        self._lights_count = lights_count
        self._lights_state = lights_state
    
    def __repr__(self) -> str:
        return f"Indicator(state={format(self._lights_state, '0'+str(self._lights_count)+'b')})"

    def __eq__(self, other) -> bool:
        return self._lights_count == other._lights_count and self._lights_state == other._lights_state

    def __hash__(self):
        return hash((self._lights_count, self._lights_state))

    @classmethod
    def create_from_string(cls, lights: str) -> Indicator:
        '''Create an instance from state string
        Args:
            lights (str): Represents state of lights in binary string format. For example: four lights, first on, three off: "1000"
        '''
        return cls(len(lights), int(lights, 2))

    def create_all_off(self) -> Indicator:
        '''Create an instance with all lights switched off
        '''
        return Indicator(self.lights_count, 0)

    def switch(self, button: Button, target_state: Indicator, applicable_buttons: list) -> Indicator:
        '''Apply button press to the lights
        Args:
            button (Button): Button compatible with the indicator
        Returns:
            tuple(Indicator, int): Updated indicator and the number of button presses required for that
        '''
        if self.lights_count != button.total_switches_count:
            raise ValueError(f"{button} is not compatible with {self}")

        return (Indicator(self.lights_count, button.control_light(self._lights_state)), 1)

    @property
    def lights_count(self) -> int:
        return self._lights_count

class JoltageLevels:
    def __init__(self, levels: list):
        '''
        Args:
            levels (list): list of integers representing the joltage levels
        '''
        self._levels = levels

    def __repr__(self) -> str:
        return f"JoltageLevels(levels={self._levels})"

    def __eq__(self, other) -> bool:
        return self._levels == other._levels

    def __hash__(self):
        return hash(tuple(self._levels))

    def create_all_zero(self) -> JoltageLevels:
        '''Create an instance with all levels set to zero
        '''
        return JoltageLevels([0] * len(self._levels))

    def switch(self, button: Button, max_levels: JoltageLevels, applicable_buttons: list) -> (JoltageLevels, int):
        '''Apply button press to the joltage levels
        Args:
            button (Button): Button compatible with the joltage levels
            max_levels (JoltageLevels): The maximum levels that can be reached
        Returns:
            tuple(JoltageLevels, int): Updated joltage levels and the number of button presses required for that
        '''
        if len(self._levels) != button.total_switches_count:
            raise ValueError(f"{button} is not compatible with {self}")

        disabled_buttons_count = 0
        max_presses = button.control_joltage(self._levels, max_levels._levels)
        levels = button.apply_joltage(self._levels, max_presses)
        for applicable_button in applicable_buttons:
            if button != applicable_button and applicable_button.control_joltage(levels, max_levels._levels) == 0:
                disabled_buttons_count += 1

        # Make as many button presses as possible without excluding possible pathes to solutions.
        # When the maximum presses disable applicable buttons (at least one), then reserve one press to keep them available.
        # But making one press is always allowed (even if it disables other button(s)).
        if disabled_buttons_count > 0:
            max_presses = max(1, max_presses - disabled_buttons_count)
            levels = button.apply_joltage(self._levels, max_presses)

        return (JoltageLevels(levels), max_presses)
    
    def get_same_level_positions(self, min_count: int) -> dict:
        '''Group levels by current value and return the list of these positions
        Args:
            min_count (int): Minimum number of same levels
        Returns:
            dict: key is the level value, while the value is a list of positions
        '''
        same_levels = {}
        for position, level in enumerate(self._levels):
            if not level in same_levels: same_levels[level] = []
            same_levels[level].append(position)
        return {level:same_levels[level] for level in same_levels if len(same_levels[level]) >= min_count}

    def get_incomplete_positions(self, other) -> set:
        '''Returns the positions which differs between the two states
        '''
        if len(self._levels) != len(other._levels):
            raise ValueError(f"{self} is not compatible with {other}")

        diff = set()
        for index in range(len(self._levels)):
            if self._levels[index] != other._levels[index]:
                diff.add(index)
        return diff

class Button:
    def __init__(self, total_switches_count: int, switches: list):
        '''
        Args:
            total_switches_count (int): Total number of switches a button could possibly control
            switches (list): Indexes of switches that this button controls (starting from zero, moving left to right)
        '''
        if total_switches_count <= 0:
            raise ValueError("Button must control at least one switch")
        if len(switches) == 0:
            raise ValueError("Button must have at least one switch configuration")

        self._total_switches_count = total_switches_count
        self._controller = 0
        self._controlled_positions = set(switches)

        for index in switches:
            if index < 0 or index >= total_switches_count:
                raise ValueError(f"Button cannot control the switch {index}. Valid switches are from 0 to {total_switches_count - 1}")

            self._controller += 2 ** (total_switches_count - index - 1)
    
    def __repr__(self) -> str:
        return f"Button(switches={format(self._controller, '0'+str(self._total_switches_count)+'b')})"

    def __lt__(self, other) -> bool:
        return len(self._controlled_positions) < len(other._controlled_positions)

    @property
    def total_switches_count(self) -> int:
        return self._total_switches_count

    def control_light(self, state: int) -> int:
        '''Apply the switches of the button to the given state of lights
        Args:
            state (int): State of lights
        Return:
            int: Updated state of lights
        '''
        return state ^ self._controller

    def control_joltage(self, levels, max_levels: list) -> int:
        '''Calculate the maximum number of button presses
        Args:
            levels (list): List of integers indicating the joltage levels indexed by position
            max_levels (list): List of integers indicating the maximum joltage levels indexed by position
        Return:
            int: Number of button presses needed to reach the maximum levels (whithout overrun)
        '''
        count = len(levels)
        max_count = len(max_levels)
        if count == 0:
            raise ValueError("'levels' must have at least one element")
        if max_count == 0:
            raise ValueError("'max_levels' must have at least one element")
        if count != max_count:
            raise ValueError("'levels' and 'max_levels' must have the same elements count")
        
        max_presses = sys.maxsize

        # find the maximum possible button presses
        for index in range(count):
            if 2 ** index & self._controller != 0:
                level_index = count - index - 1
                max_presses = min(max_presses, max(0, max_levels[level_index] - levels[level_index]))

        return max_presses
    
    def apply_joltage(self, levels: list, presses: int) -> list:
        '''Apply the given number of button presses to the levels
        Args:
            levels (list): List of integers indicating the joltage levels indexed by position
            presses (int): Number of button presses
        Return:
            list: Updated joltage levels
        '''
        if presses <= 0:
            raise ValueError("'presses' must be a positive interger")
        
        updated_levels = levels.copy()
        count = len(updated_levels)
        
        for index in range(count):
            if 2 ** index & self._controller != 0:
                level_index = count - index - 1
                updated_levels[count - index - 1] += presses
        
        return updated_levels

    def has_only_positions(self, positions: set) -> bool:
        '''True when the button controls some or all of the given positions
        '''
        return len(self._controlled_positions.difference(positions)) == 0

    def is_subset_of(self, other) -> bool:
        '''True when this button controls all positions of the other
        '''
        return self._controlled_positions.issubset(other._controlled_positions)

class ButtonSelectorInterface:
    def applicable_buttons(state) -> list:
        '''Return a list of applicable buttons based on the given state
        '''
        raise NotImplementedError("ButtonSelectorInterface must be implemented")

class IndicatorAwareButtonSelector(ButtonSelectorInterface):
    def __init__(self, buttons: list):
        self._buttons = buttons

    def applicable_buttons(self, state) -> list:
        '''All buttons can be used in any Indicator state
        '''
        return self._buttons
    
class JoltageLevelAwareButtonSelector(ButtonSelectorInterface):
    def __init__(self, buttons: list, target_state: JoltageLevels):
        self._buttons = buttons
        self._target_state = target_state

    def applicable_buttons(self, state: JoltageLevels) -> list:
        '''Filter and return the buttons that bring closer to the target state
        
        1. Exclude buttons which would further increase the joltage that are already at the expected level.
        
        Example: state = [2, 3, 2] target = [3, 3, 3] buttons = [ (0,1,2), (0,2) ]
        The button (0,1,2) would increment the middle meter which is already at level 3, so this should be excluded.

        2. Exclude symmetric pathes in the state graph due to commutative property of addition.
        
        ExampleA: state = [0, 0, 0] target = [3, 2, 4] buttons = [ (0), (1), (2) ]
        All of the four buttons are applicable in this state, but it does not matter which button to start with, as all ends up in the same target state.
        So let's choose the first one (0) and ignore the rest (1) and (2). This cuts symmetric solutions.
        
        ExampleB: state = [0, 0, 0] target = [3, 2, 4] buttons = [ (0), (1), (2), (0,1,2) ]
        In this setup the last button covers the first three with one step instead of three.
        So let's choose the biggest button (0,1,2) and ignore (0), (1) and (2).
        '''
        # 1. case
        incomplete_positions = self._target_state.get_incomplete_positions(state)
        filtered_buttons = [button for button in self._buttons if button.has_only_positions(incomplete_positions)]

        print(f"for {state} the applicable buttons are: {filtered_buttons}")

        # 2. case
        for positions in state.get_same_level_positions(min_count=2).values():
            incomplete_same_level_positions = [position for position in positions if position in incomplete_positions]
            covering_buttons = [(button.total_switches_count, button) for button in filtered_buttons if button.has_only_positions(incomplete_same_level_positions)]
            for _, button in sorted(covering_buttons)[:-1]:
                filtered_buttons.remove(button)

        print(f"... after removal of same level buttons: {filtered_buttons}\n")
        return filtered_buttons

def compute_minimum_button_presses(start_state, target_state, button_selector: ButtonSelectorInterface) -> int:
    g = nx.Graph()
    g.add_node(start_state, processed=False)
    g.add_node(target_state, processed=True)

    while True:
        # take any unprocessed state
        try:
            state = next(node for node, attributes in g.nodes(data=True) if not attributes["processed"])
        except StopIteration:
            break

        # generate new states by pressing buttons
        applicable_buttons = button_selector.applicable_buttons(state)
        for button in applicable_buttons:
            new_state, presses = state.switch(button, target_state, applicable_buttons)
            #print(f"{state} -> {new_state} by pressing {button} {presses} times")
            if not new_state in g.nodes:
                g.add_node(new_state, processed=False)
            g.add_edge(state, new_state, presses=presses)

        g.nodes[state]["processed"] = True

    # find the shortest path in the state graph
    return nx.shortest_path_length(g, start_state, target_state, weight="presses")

def create_indicator(config: str) -> Indicator:
    return Indicator.create_from_string(config.replace('.', '0').replace('#', '1'))

def create_buttons(lights_count: int, config: str) -> list:
    buttons = []
    for button_config in config.strip().split(' '):
        light_switches = list(map(int, button_config.strip('()').split(',')))
        buttons.append(Button(lights_count, light_switches))
    return buttons

def create_joltage_levels(config: str) -> JoltageLevels:
    return JoltageLevels(list(map(int, config.strip().split(','))))

def sum_of_minimal_button_presses(target_states, buttons: list, create_start_state, create_button_selector: callable) -> int:
    count = 0
    for input_index, target_state in enumerate(target_states):
        print(f"calculating min button presses for {input_index}...")
        count += compute_minimum_button_presses(create_start_state(target_state), target_state, create_button_selector(buttons[input_index], target_state))
        print(f" count={count} done")
    return count

target_indicators = []
buttons = []
target_joltage_levels = []
with input_file.open(mode="r", encoding="utf-8") as file:
    for line in file:
        matches = re.match(r"\[(.+)\] ([^{}]+) {(.+)}", line)
        indicator_config = matches[1]
        buttons_config = matches[2]
        joltage_config = matches[3]
        indicator = create_indicator(indicator_config)
        target_indicators.append(indicator)
        buttons.append(create_buttons(indicator.lights_count, buttons_config))
        target_joltage_levels.append(create_joltage_levels(joltage_config))

create_start_state = lambda indicator: indicator.create_all_off()
create_button_selector = lambda buttons, target_state: IndicatorAwareButtonSelector(buttons)
#print(f"Part1 answer: {sum_of_minimal_button_presses(target_indicators, buttons, create_start_state, create_button_selector)}")

create_start_state = lambda joltage_levels: joltage_levels.create_all_zero()
create_button_selector = lambda buttons, target_state: JoltageLevelAwareButtonSelector(buttons, target_state)
print(f"Part2 answer: {sum_of_minimal_button_presses(target_joltage_levels, buttons, create_start_state, create_button_selector)}")


calculating min button presses for 0...
for JoltageLevels(levels=[0, 0, 0, 0]) the applicable buttons are: [Button(switches=0111), Button(switches=1010), Button(switches=1100), Button(switches=0011)]
... after removal of same level buttons: [Button(switches=0111)]

for JoltageLevels(levels=[0, 3, 3, 3]) the applicable buttons are: [Button(switches=1010), Button(switches=1100)]
... after removal of same level buttons: [Button(switches=1010), Button(switches=1100)]

for JoltageLevels(levels=[3, 3, 6, 3]) the applicable buttons are: [Button(switches=1100)]
... after removal of same level buttons: [Button(switches=1100)]

for JoltageLevels(levels=[2, 5, 3, 3]) the applicable buttons are: [Button(switches=1010)]
... after removal of same level buttons: [Button(switches=1010)]

for JoltageLevels(levels=[5, 5, 6, 3]) the applicable buttons are: []
... after removal of same level buttons: []



NetworkXNoPath: Node JoltageLevels(levels=[6, 5, 6, 3]) not reachable from JoltageLevels(levels=[0, 0, 0, 0])

In [71]:
lb1 = Button(4, [3])
lb2 = Button(4, [2,3])
lb3 = Button(4, [0,1])

i1 = Indicator("0110")
i2 = Indicator("0100")
i3 = Indicator("1000")
i4 = Indicator("1111")

print(i1, lb1, lb2, lb3)
i1.switch(lb1)
print(i1)
i1.switch(lb1)
print(i1, "\n")

i1.switch(lb3)
print(i1)
i1.switch(lb2)
print(i1)
i1.switch(lb1)
print(i1)

Indicator(state=0110) LightButton(switches=0001) LightButton(switches=0011) LightButton(switches=1100)
Indicator(state=0111)
Indicator(state=0110) 

Indicator(state=1010)
Indicator(state=1001)
Indicator(state=1000)


2

In [8]:
j1 = JoltageLevels([1,2,3,4])
j2 = JoltageLevels([1,2,3,4])
g = nx.Graph()
g.add_node(j1)
print(j1 in g.nodes, j2 in g.nodes, j1 == j2)
g.add_node(j2)

iterator = iter(n for n in g.nodes)

for n in iterator:
    print(n)

#nx.shortest_path_length(g, 4, 1)

True False False
JoltageLevels(levels=[1, 2, 3, 4])
JoltageLevels(levels=[1, 2, 3, 4])


In [13]:
i1 = Indicator.create_from_string("0110")
i1_copy = Indicator.create_from_string("0110")
i2 = Indicator.create_from_string("0100")
g = nx.Graph()
g.add_node(i1, custom=True)
g.nodes[i1]["custom"] = False

print(i1 in g.nodes, i1_copy in g.nodes, i1 == i1_copy)
g.nodes[i1]["custom"]

True True True


False

In [100]:
b1 = Button(4, [1, 3])
b2 = Button(4, [0, 2])
j = JoltageLevels([3, 5, 4 ,7])

print(j.switch(b1).switch(b2))

print(j.create_all_zero())

JoltageLevels(levels=[4, 6, 5, 8])
JoltageLevels(levels=[0, 0, 0, 0])


In [38]:
filtered_buttons = sorted([Button(4, [0,1,3]), Button(4, [0]), Button(4, [2]), Button(4, [0,2]), Button(4, [1,3])])

merged_buttons = filtered_buttons.copy()
for index1, button1 in enumerate(filtered_buttons):
    for index2 in range(index1 + 1, len(filtered_buttons)):
        if button1.is_subset_of(filtered_buttons[index2]):
            merged_buttons.remove(button1)
            break

print(merged_buttons)

[Button(switches=1000), Button(switches=0010), Button(switches=1010), Button(switches=0101), Button(switches=1101)]
[Button(switches=1010), Button(switches=1101)]
